## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Importing Necessary libraries
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, ArrayType
from pyspark.sql.functions import current_timestamp, input_file_name, desc, explode, to_timestamp, when, col, udf

from pyspark.ml.feature import Tokenizer
from pyspark.ml import PipelineModel

import mlflow
import mlflow.sklearn
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from delta.tables import DeltaTable 

In [0]:
# Setting shuffle partition in multiple of 4
spark.conf.set("spark.sql.shuffle.partitions", 16)

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# 2.1 Reading the sourse file directory listing
data = get_source_listing_df()
data

In [0]:
# 2.2 Counting the number of source files
print(len(data))

In [0]:
# 2.3 Printing the contents of a file
show_s3_file_contents("voc_volume/1.json")

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# 3.1 Defining the schema for the raw data
bronze_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time", StringType(), True)
])

In [0]:
# 3.2 Setting a read stream using cloudfiles and the source data format
# 3.5 Allow a new schema to be merged into the bronze delta table

raw_input_data = (spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .schema(bronze_schema)
    .option("mergeSchema", "true")
    .load(TWEET_SOURCE_PATH)
    
)

# 3.8 Adding source_file and processing time columns
transformed_raw_data = (raw_input_data
    .withColumn("source_file", input_file_name())
    .withColumn("processing_time", current_timestamp())
)

In [0]:
# 3.3 Setting a write stream using cloudfiles to append to the bronze delta table
# 3.4 Enforce schema
# 3.6 Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
# 3.7 Name your raw to bronze stream as bronze_stream

bronze_write_stream = (transformed_raw_data
    .writeStream
    .outputMode("append")
    .format("delta")
    .queryName("bronze_stream")
    .option("enforceSchema", "true")
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .trigger(availableNow = True)
    .start(BRONZE_DELTA)
)

In [0]:
bronze_write_stream.awaitTermination()

In [0]:
%sql
DROP TABLE IF EXISTS bronze_temp_view;

In [0]:
bronze_temp = (spark
  .readStream
  .load(BRONZE_DELTA)
  .createOrReplaceTempView("bronze_temp_view"))

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
%sql
-- 4.1 Computing the number of tweets captured in the bronze table
SELECT COUNT(*) FROM bronze_temp_view

In [0]:
%sql
-- 4.2 Counting Null or Nan data in each column of the raw data
SELECT 
    count_if(date IS NULL OR isnan(date)) AS date_null_count,
    count_if(user IS NULL OR isnan(user)) AS user_null_count,
    count_if(text IS NULL OR isnan(text)) AS text_null_count,
    count_if(sentiment IS NULL OR isnan(sentiment)) AS sentiment_null_count
FROM 
    bronze_temp_view;

In [0]:
%sql

-- 4.3 Count the number of tweets by each unique user handle and sort the data by descending count

CREATE OR REPLACE TEMPORARY VIEW user_tweet_count AS
SELECT user, count(*) as tweet_count
FROM bronze_temp_view
GROUP BY user
ORDER BY tweet_count DESC;

In [0]:
%sql
-- 4.4 How many tweets have at least one mention (@) how many tweet have no mentions (@)

-- 4.4a Tweet count with atleast one mention
SELECT COUNT(*) AS at_least_one_mentions_count
FROM bronze_temp_view
WHERE text LIKE '%@%';

In [0]:
%sql
-- 4.4b Tweet count with no mentions
SELECT COUNT(*) AS no_mentions_count
FROM bronze_temp_view
WHERE text NOT LIKE '%@%';

In [0]:
%sql

-- 4.5 Plotting bar chart that shows the top 20 tweeters (users)
SELECT * FROM user_tweet_count
ORDER BY tweet_count DESC
LIMIT 20

Databricks visualization. Run in Databricks to view.

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# 5.1 Setting a read stream on the bronze delta table
bronze_table_data = (spark
    .readStream
    .load(BRONZE_DELTA)
)

In [0]:
## 5.5 Bronze to Silver Transformations

# UDF to extract mentions starting with '@' from text
extract_mentions = udf(lambda text: [word[1:] for word in text.split() if word.startswith('@') and len(word) > 1], ArrayType(StringType()))

# UDF to extract cleaned text
extract_cleaned_text = udf(lambda text: ' '.join([word for word in text.split() if not word.startswith('@')]))

# Filtering rows with having atleast one mentions
# Adding 'mention' and 'cleaned_text' columns using udf
bronze_to_silver = (bronze_table_data
    .select("date", "text", "sentiment")
    .filter(col("text").like("%@%"))
    .withColumn("mention_array", extract_mentions(col("text")))
    .withColumn("cleaned_text", extract_cleaned_text(col("text")))
)

# Explode the mention array column to get the row for each mention
bronze_to_silver = (bronze_to_silver
    .select("*", explode("mention_array").alias("mention"))
)

# Adding timestamp column using date column
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
bronze_to_silver = bronze_to_silver.withColumn("timestamp", to_timestamp(col("date"), "EEE MMM dd HH:mm:ss z yyyy"))

# Dropping the redundant columns
bronze_to_silver = bronze_to_silver.drop("date", "text", "mention_array")


In [0]:
display(bronze_to_silver)

In [0]:
# 5.2 Setting the write stream to append to the silver delta table
# 5.3 Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
# 5.4 Name your bronze to silver stream as silver_stream

silver_write_stream = (bronze_to_silver
    .writeStream
    .outputMode("append")
    .format("delta")
    .queryName("silver_stream")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .trigger(availableNow = True)
    .start(SILVER_DELTA)
)

In [0]:
silver_write_stream.awaitTermination()

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# 6.1 Setting read stream of the Silver delta table

silver_table_data = (spark
    .readStream
    .load(SILVER_DELTA)
)

In [0]:
silver_table_data.createOrReplaceTempView("silver_table_temp")

In [0]:
%sql
-- Silver Table count
SELECT COUNT(*) FROM silver_table_temp

In [0]:

# 6.6 Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry

model = mlflow.pyfunc.spark_udf(spark, "models:/HF_TWEET_SENTIMENT/Production")

In [0]:
# 6.5 transform the silver data to the gold data using the data definition at the top of the notebook
# 6.7 Use a spark UDF to parallelize the inference across your silver data

silver_to_gold = silver_table_data.withColumn("predictions", model(col("cleaned_text")))

silver_to_gold = (silver_to_gold
    .withColumn("predicted_score", col("predictions.score") * 100)
    .withColumn("predicted_sentiment", col("predictions.label"))
    .withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(0))
    .withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "POS", 1).when(col("predicted_sentiment") == "NEG", 0).otherwise(-1))
    .select("timestamp", "mention", "cleaned_text", "sentiment", "predicted_score", "predicted_sentiment", "sentiment_id", "predicted_sentiment_id")
)

In [0]:
display(silver_to_gold)

In [0]:
# 6.2 Setting the write stream to append to the silver delta table
# 6.3 Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
# 6.4 name your silver to gold stream as gold_stream

gold_write_stream = (silver_to_gold
    .writeStream
    .outputMode("append")
    .format("delta")
    .queryName("gold_stream")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .trigger(availableNow = True)
    .start(GOLD_DELTA)
)

In [0]:
gold_write_stream.awaitTermination()

In [0]:
%sql
DROP TABLE IF EXISTS gold_table_temp;

In [0]:

gold_table_data = (spark
    .readStream
    .format("delta")
    .load(GOLD_DELTA)
    .createOrReplaceTempView("gold_table_temp")
)

In [0]:
%sql
-- Counting number of rows in Gold Table 
SELECT COUNT(*) FROM gold_table_temp

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
gold_table = spark.read.format("delta").load(GOLD_DELTA)
# display(gold_table)

In [0]:
with mlflow.start_run(run_name = "TweetSentimentModel") as run:
    class_labels = ["neutral", "negative", "positive"] # -1, 0 and 1 respectively

    actual_sentiments = gold_table.select("sentiment_id").toPandas()["sentiment_id"]
    predicted_sentiments = gold_table.select("predicted_sentiment_id").toPandas()["predicted_sentiment_id"]
    predicted_sentiments = predicted_sentiments.fillna(-1)

    # Calculate precision, recall, and F1-score
    precisions = precision_score(actual_sentiments, predicted_sentiments, average=None)
    recalls = recall_score(actual_sentiments, predicted_sentiments, average=None)
    f1_scores = f1_score(actual_sentiments, predicted_sentiments, average=None)

    # 7.1 Log precision, recall, and F1-score as MLflow metrics
    for sentiment, precision, recall, f1 in zip(class_labels, precisions, recalls, f1_scores):
        mlflow.log_metric(f"precision_{sentiment}", precision)
        mlflow.log_metric(f"recall_{sentiment}", recall)
        mlflow.log_metric(f"f1_score_{sentiment}", f1)

    # Create the confusion matrix
    confusion_matrix = pd.crosstab(actual_sentiments, predicted_sentiments, dropna = True)

    # Plot and save the confusion matrix as an image
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_matrix, annot=True, cmap="Blues", fmt="d")
    plt.xlabel("Predicted Sentiment")
    plt.ylabel("Actual Sentiment")
    plt.savefig("confusion_matrix.png")

    # 7.2 Log the confusion matrix image as an MLflow artifact
    mlflow.log_artifact("confusion_matrix.png")

    # 7.3 Log the model name and the MLflow version that was used as an MLflow parameters
    mlflow.log_param("Model Name", MODEL_NAME)
    mlflow.log_param("Model Version", "1")
    
    delta_table = DeltaTable.forPath(spark, SILVER_DELTA)
    latest_silver_version = delta_table.history().select("version").limit(1).collect()[0][0]

    # 7.4 Log the version of the Delta Table (input-silver) as an MLflow parameter
    mlflow.log_param("Silver Table Version", str(latest_silver_version))

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
%sql

-- 8.1 How many mentions are there in the gold data total?

SELECT COUNT(mention) FROM gold_table_temp

In [0]:
%sql
-- 8.2 Count the number of neutral, positive and negative tweets for each mention in new columns
-- 8.3 Capture the total for each mention in a new column
-- 8.4 Sort the mention count totals in descending order

CREATE OR REPLACE TEMPORARY VIEW user_tweet_type_count AS
SELECT
  mention,
  SUM(CASE WHEN predicted_sentiment_id = -1 THEN 1 ELSE 0 END) AS neutral_count,
  SUM(CASE WHEN predicted_sentiment_id = 1 THEN 1 ELSE 0 END) AS positive_count,
  SUM(CASE WHEN predicted_sentiment_id = 0 THEN 1 ELSE 0 END) AS negative_count,
  COUNT(*) AS total_count
FROM
  gold_table_temp
GROUP BY
  mention
ORDER BY
  total_count DESC;

In [0]:
%sql
-- 8.2, 8.3 and 8.4 result / output
SELECT * FROM user_tweet_type_count;

In [0]:
%sql
-- 8.5 Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)

SELECT mention, positive_count FROM user_tweet_type_count
ORDER BY positive_count DESC
LIMIT 20;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- 8.6 Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

SELECT mention, negative_count FROM user_tweet_type_count
ORDER BY negative_count DESC
LIMIT 20;

Databricks visualization. Run in Databricks to view.

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# 9.1 Printing running Streams
for stream in spark.streams.active:
    print(stream.name)

# 9.2 Stop all active streams
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

# 9.3 Printing notebook elapsed time
print("Elapsed Time:", END_TIME - START_TIME)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

**1. Spill:**

Observation: There are no signs of data spill in the system. This observation is supported by the uniform size of partitions across jobs and the relatively small file sizes at the source.

Implication: Since there are no large files causing disproportionate loading or processing, performance issues related to disk spill are effectively mitigated.

**2. Skew:**

Observation: Data skew is absent within the job executions. The uniformity in file sizes contributes to this balance. Additionally, Adaptive Query Execution (AQE) optimizes the partitions, which further alleviates potential skew.

Implication: The balanced distribution ensures that no single node or process becomes a bottleneck due to uneven data loads.

**3. Shuffle:**

Observation: Shuffle operations are prevalent due to necessary wide transformations such as GroupBy, OrderBy, Aggregate, and Explode.
Optimization: By setting the number of shuffle partitions to be a multiple of the number of cores, the data movement across partitions is optimized, minimizing potential delays in shuffle operations.

Implication: While shuffle is inherent to the operations being performed, the strategic configuration of shuffle partitions reduces the time overhead associated with these operations.

**4. Storage:**

Observation: The initial storage of JSON files in the S3 bucket is suboptimal due to the presence of numerous small files, leading to inefficient read times.

Further Steps: Post initial read, data is efficiently stored in the Bronze, Silver, and Gold Delta tables, which suggests that improvements should focus on the initial ingestion phase, possibly by consolidating files before processing or adjusting the read mechanisms.

**5. Serialization:**

Observation: The serialization of the ML model for sentiment analysis, implemented as a User Defined Function (UDF), represents the most time-consuming operation in the project.

Alternative Considerations: Although replacing this UDF is not feasible, optimizing other UDFs, such as the one used for extracting mentions in the Silver table, could yield performance benefits. Exploring built-in Spark functions that perform similar tasks could reduce the reliance on custom UDFs, which are generally less efficient.